In [1]:
import os.path
import sys
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
import scoping
from scoping.models import *
from django.core.exceptions import ObjectDoesNotExist, MultipleObjectsReturned

import pandas as pd

ModuleNotFoundError: No module named 'django'

In [17]:
q = Query.objects.get(pk=10997)
p = q.project

dids = set(Doc.objects.filter(query__project=p).values_list('pk', flat=True))
p.id

338

In [18]:
#if pd.read_csv('../data/0c_doc_info.csv').shape[0] != len(dids):
df = pd.DataFrame.from_dict(Doc.objects.filter(pk__in=dids).values('id','PY','title','wosarticle__dt'))
df.head()
df.to_csv('data/0c_doc_info.csv',index=False)

In [28]:
rels = Doc.objects.filter(docproject__project=p, docproject__relevant=1)
for d in rels:
    dos = d.docownership_set.filter(query__project=p)
    if not dos.exists():
        break

In [31]:
d.title

'The effectiveness of a computer-based method to support eating intervention among economically disadvantaged children in Malaysia'

In [21]:
import statistics

django.db.connection.close()

def mode_na(x):
    try:
        return statistics.mode([y for y in x if y is not np.NaN])
    except:
        return np.NaN
    
columns = [
    'id',
    'content',
    'title',
    'tslug',
    'wosarticle__de',
    'wosarticle__wc'
]

# Majority ratings
dos = DocOwnership.objects.filter(
    query__project=p, 
    relevant__gt=0,
    #doc__content__iregex='\w'
)

dodf = pd.DataFrame.from_dict(dos.order_by('finish').values(
    'id','doc_id','user__username','relevant'
))
dodf.loc[dodf['relevant']==2,'relevant'] = 0
dodf.loc[dodf['relevant']==3,'relevant'] = 0

seen_df = dodf.groupby('doc_id').agg(lambda x: mode_na(x)).reset_index()[['doc_id','relevant']].rename(columns={"doc_id":"id"})
seen_df['seen'] = 1
seen_df.loc[pd.isna(seen_df['relevant']),'seen'] = np.NaN

ddf = pd.DataFrame.from_dict(
    Doc.objects.filter(pk__in=seen_df['id']).values(*columns)
)
seen_df = pd.merge(ddf,seen_df)

print(seen_df.shape)
print(seen_df.relevant.sum())
seen_df.head()

(3084, 8)
144


,id,content,title,tslug,wosarticle__de,wosarticle__wc,relevant,seen
0,5386387,Little is known about how peers influence the ...,When incentives backfire: Spillover effects in...,whenincentivesbackfirespillovereffectsinfoodch...,Business And Economics; Peers; Field study; Sn...,None,0,1
1,5386550,At a time when policy makers want to change th...,Nudging Healthy Lifestyles Informing Regulator...,nudginghealthylifestylesinformingregulatorygov...,None,None,0,1
2,1294832,This paper presents a pragmatic framework to i...,A pragmatic framework to score and inform abou...,apragmaticframeworktoscoreandinformabouttheenv...,Life cycle assessment; Nutrition; Diet; Ecolog...,[Green & Sustainable Science & Technology; Eng...,0,1
3,5386412,"For products with risky characteristics, there...",Consumer Acceptance of Traffic-light Labelling...,consumeracceptanceoftrafficlightlabellingonfoo...,Business And Economics; Consumers; Food; ideas...,None,0,1
4,4388598,Waste is an unused raw material or energy sour...,Integrated waste management system for resort ...,integratedwastemanagementsystemforresorttown,None,None,0,1


In [24]:
seen_df.relevant.unique()

array([0, 1])

In [5]:
seen_df.to_csv('data/0_labelled_documents.csv',index=False)
print(seen_df.relevant.sum())
seen_df.head()

144


,id,content,title,tslug,wosarticle__de,wosarticle__wc,relevant,seen
0,5386387,Little is known about how peers influence the ...,When incentives backfire: Spillover effects in...,whenincentivesbackfirespillovereffectsinfoodch...,Business And Economics; Peers; Field study; Sn...,None,0,1.0
1,5386550,At a time when policy makers want to change th...,Nudging Healthy Lifestyles Informing Regulator...,nudginghealthylifestylesinformingregulatorygov...,None,None,0,1.0
2,5343582,This study calculated the carbon footprint of ...,An assessment of the carbon footprint of resta...,anassessmentofthecarbonfootprintofrestaurantsb...,None,None,0,1.0
3,1294832,This paper presents a pragmatic framework to i...,A pragmatic framework to score and inform abou...,apragmaticframeworktoscoreandinformabouttheenv...,Life cycle assessment; Nutrition; Diet; Ecolog...,[Green & Sustainable Science & Technology; Eng...,0,1.0
4,4388598,Waste is an unused raw material or energy sour...,Integrated waste management system for resort ...,integratedwastemanagementsystemforresorttown,None,None,0,1.0


In [6]:
columns = [
    'id',
    'content',
    'title',
    'tslug',
    'wosarticle__de',
    'wosarticle__wc'
]

early_docs = set(DocOwnership.objects.filter(query__project=p,tag__id__lt=6537).values_list('doc_id',flat=True))

unseen_docs = Doc.objects.filter(
    id__in=dids,
    content__iregex='\w',
)#.exclude(pk__in=set(seen_df['id'])-early_docs).distinct('id')

unseen_df = pd.DataFrame.from_dict(list(unseen_docs.values(*columns)))
unseen_df = unseen_df[columns]
unseen_df['relevant'] = 0
unseen_df['seen'] = 0
unseen_df['random_sample'] = 0

print(unseen_df.shape)


(28583, 9)


In [7]:
duplicateRowsDF = unseen_df[unseen_df.duplicated(['tslug'])]
duplicateRowsDF.shape

(55, 9)

In [8]:
def doc_merge(request,pid: int, d1: int, d2: int) -> str:
    """Merge duplicated documents

Ajax function

Adds all the info from the given project from the document 2 to document 1,
removing it from all queries

Args:
    * pid (int): Project ID
    * d1 (int): ID of Document to keep
    * d2 (int): ID of Document to throw

Todos:
    * Remove docproject object?

..

    """
    p = Project.objects.get(pk=pid)
    keep = Doc.objects.get(pk=d1)
    throw = Doc.objects.get(pk=d2)
    dup, created = Duplicate.objects.get_or_create(
        original = keep,
        copy = throw,
        project = p,
        user = request.user
    )
    if keep.alternative_titles is None:
        keep.alternative_titles = [throw.title]
    elif throw.title not in keep.alternative_titles:
        keep.alternative_titles.append(throw.title)
    for q in throw.query.filter(project=p):
        throw.query.remove(q)
        keep.query.add(q)
        q.r_count = q.doc_set.count()
        q.save()
    for t in throw.tag.filter(query__project=p):
        throw.tag.remove(t)
        keep.tag.add(t)
        t.update_tag()
    models = [
        StudyEffect,DocMetaCoding,DocProject,
        Exclusion, DocUserCat, DocCat,
        DocPar, DocOwnership
        ## Need to filter by projectt!!!
    ]
    print("DOCOWNERSHIP")
    
    for x in models:
        if hasattr(x,'project'):
            if hasattr(x, 'query'):
                insts = x.objects.filter(doc=throw, query__project=p) | x.objects.filter(doc=throw, project=p)
            else:
                insts = x.objects.filter(doc=throw,project=p)
        elif hasattr(x,'category'):
            insts = x.objects.filter(doc=throw,category__project=p)
        else:
            insts = []
            print(x)
        for i in insts:
            print(i)
            ignore_fields = ["_state","doc__id","doc_id","doc","id"]
            instance = {k:v for k,v in i.__dict__.items() if k not in ignore_fields}
            instance["doc"] = keep
            try:
                new_int, created = x.objects.get_or_create(**instance)
            except django.db.utils.IntegrityError:
                print("already exists")
            i.delete()
    # Return a message saying "merged"
    return "merged"

class Request:
    def __init__(self):
        self.user = User.objects.get(pk=1)
    
request = Request()

deduplicate = False
if deduplicate:
    for name, group in duplicateRowsDF.groupby('tslug'):
        docs = Doc.objects.filter(tslug=name)
        print(name)
        try:
            d1 = docs.get(UT__UT__contains="WOS:")
        except:
            d1 = None
        if d1 is None:
            continue

        for d2 in docs.exclude(pk=d1.pk):
            doc_merge(request, p.id, d1.id, d2.id)

In [9]:
for q in p.query_set.all():
    q.r_count = q.doc_set.count()
    q.save()

In [10]:
unseen_df.to_csv('data/0_unlabelled_documents.csv', index=False)
unseen_df.head()

,id,content,title,tslug,wosarticle__de,wosarticle__wc,relevant,seen,random_sample
0,891,Harvesting of wild-living animals is often int...,Fast and behavior-selective exploitation of a ...,fastandbehaviorselectiveexploitationofamarinef...,None,[Multidisciplinary Sciences],0,0,0
1,1406,This paper presents the results of the cradle ...,Cradle to retailer or quick service restaurant...,cradletoretailerorquickservicerestaurantgateli...,Life cycle assessment; Chicken; Free-range; Co...,[Green & Sustainable Science & Technology; Eng...,0,0,0
2,2124,Oilseed crops are the basis for biological sys...,Breeding Oilseed Crops for Climate Change,breedingoilseedcropsforclimatechange,None,None,0,0,0
3,2144,While both academics and politicians have long...,Feeding unrest: Disentangling the causal relat...,feedingunrestdisentanglingthecausalrelationshi...,Africa; food prices; food security; instrument...,[International Relations; Political Science],0,0,0
4,4215,The increasing share of imported food in devel...,Chinese producer behavior: Aquaculture farmers...,chineseproducerbehavioraquaculturefarmersinsou...,Agricultural economics; Aquaculture; Producers...,[Economics],0,0,0
